# stackoverflow

https://stackoverflow.com/questions/42752610/python-how-to-generate-the-pairwise-hamming-distance-matrix

In [12]:
import numpy as np

In [13]:
arr= np.array([[ 1,  0,  0,  1,  1,  0],
   [ 1,  0,  0,  0,  0,  0],
   [ 1,  1,  1,  1,  0,  0],])

In [14]:
arr

array([[1, 0, 0, 1, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 0, 0]])

In [15]:
arr.itemsize

8

In [16]:
(arr[:, None, :] != arr).sum(2)

# array([[0, 2, 3],
#        [2, 0, 3],
#        [3, 3, 0]])
def compute_HammingDistance(X):
    return (X[:, None, :] != X).sum(2)

In [17]:
compute_HammingDistance(arr)

array([[0, 2, 3],
       [2, 0, 3],
       [3, 3, 0]])

# use bool

https://stackoverflow.com/questions/42752610/python-how-to-generate-the-pairwise-hamming-distance-matrix

In [19]:
arr = arr.astype(bool)

In [20]:
arr

array([[ True, False, False,  True,  True, False],
       [ True, False, False, False, False, False],
       [ True,  True,  True,  True, False, False]])

In [21]:
arr.itemsize

1

In [22]:
(arr[:, None, :] != arr).sum(2)

# array([[0, 2, 3],
#        [2, 0, 3],
#        [3, 3, 0]])
def compute_HammingDistance(X):
    return (X[:, None, :] != X).sum(2)

In [23]:
compute_HammingDistance(arr)

array([[0, 2, 3],
       [2, 0, 3],
       [3, 3, 0]])

# setup

In [24]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from efficientnet_pytorch import EfficientNet
from PIL import Image
from torchvision import transforms

from transformers import AutoModel, AutoTokenizer
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
import torch, gc

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

HOME = '/data/git/shopee-product-matching'
pdata = f'{HOME}/data/shopee-product-matching'
!ls $pdata

BS = 256
NWKRS = 8
DEVICE = 'cuda'
PIN_MEMORY = True
MAXLEN = 128

sample_submission.csv  test.csv  test_images  train.csv  train_images


# data

In [25]:
fnm = 'train'
df = pd.read_csv(f'{pdata}/{fnm}.csv')
assert len(df) == df.posting_id.nunique()
p_imgs = f"{pdata}/train_images"
df.head(2)

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045


# hash

In [26]:
def hamming(x1, x2): 
    x1,x2 = (bin(int(x, 16)) for x in (x1, x2))
    return sum(o1!=o2 for o1,o2 in zip(x1,x2))
n = 5
hdists = np.zeros((n, n))
for i in tqdm(range(n)):
    for j in range(i, n):
        x1 = df.image_phash[i]
        x2 = df.image_phash[j]
        hdists[i,j]=hdists[j,i]=hamming(x1, x2)
hdists

100%|██████████| 5/5 [00:00<00:00, 6311.02it/s]


array([[ 0., 42., 40., 30., 32.],
       [42.,  0., 28., 28., 34.],
       [40., 28.,  0., 34., 44.],
       [30., 28., 34.,  0., 34.],
       [32., 34., 44., 34.,  0.]])

In [27]:
hfeats = df[:5].image_phash.apply(lambda x: bin(int(x, 16))).values

In [28]:
X = np.array([[eval(o) for o in x[2:]] for x in hfeats])
X

array([[1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1,
        1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1],
       [1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1,
        0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1],
       [1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0,
        0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1],
       [1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,
        0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,

In [29]:
X = X.astype(bool)

In [30]:
compute_HammingDistance(X)

array([[ 0, 42, 40, 30, 32],
       [42,  0, 28, 28, 34],
       [40, 28,  0, 34, 44],
       [30, 28, 34,  0, 34],
       [32, 34, 44, 34,  0]])

# all data

In [32]:
hfeats = df.image_phash.apply(lambda x: bin(int(x, 16))).values

In [33]:
%%time
X = np.array([[eval(o) for o in x[2:]] for x in hfeats], dtype=np.bool)


CPU times: user 8.79 s, sys: 11.9 ms, total: 8.8 s
Wall time: 8.8 s


In [34]:
%%time
hdists = compute_HammingDistance(X)

CPU times: user 4min 5s, sys: 2min 14s, total: 6min 19s
Wall time: 12min 58s


# demo

In [48]:
hsims = 1/(1+hdists)

In [49]:
hsims

array([[1.        , 0.02325581, 0.02439024, ..., 0.02702703, 0.02857143,
        0.03225806],
       [0.02325581, 1.        , 0.03448276, ..., 0.03703704, 0.03030303,
        0.04347826],
       [0.02439024, 0.03448276, 1.        , ..., 0.03225806, 0.02564103,
        0.02857143],
       ...,
       [0.02702703, 0.03703704, 0.03225806, ..., 1.        , 0.03448276,
        0.03448276],
       [0.02857143, 0.03030303, 0.02564103, ..., 0.03448276, 1.        ,
        0.02857143],
       [0.03225806, 0.04347826, 0.02857143, ..., 0.03448276, 0.02857143,
        1.        ]])

# demo

In [68]:
i = np.random.choice(range(len(df)))
print(i)
df.iloc[i].to_dict()

25418


{'posting_id': 'train_3310166485',
 'image': 'be2ed83862260658093ee0cf72effc6e.jpg',
 'image_phash': 'a179841fd6075c79',
 'title': 'Dispenser Sabun Cair 2 in 1 With Key Lock 2 Tabung',
 'label_group': 2641040768}

In [69]:
grp = df.iloc[i].label_group

df[df.label_group==grp][['title', 'image_phash', 'label_group']]

,title,image_phash,label_group
25418,Dispenser Sabun Cair 2 in 1 With Key Lock 2 Ta...,a179841fd6075c79,2641040768
31294,Dispenser Sabun Cair 2 in 1 With Key Lock 2 Ta...,f9698c9e86868e39,2641040768


In [70]:
ntop = 10

In [71]:
idx_sim = np.argsort(-hsims[i])[:ntop]
df.iloc[idx_sim][['title', 'image_phash', 'label_group']]

,title,image_phash,label_group
25418,Dispenser Sabun Cair 2 in 1 With Key Lock 2 Ta...,a179841fd6075c79,2641040768
15338,SEPASANG EMBLEM STIKER TIMBUL YAMAHA RX KING,ede1861f960cd919,2798182857
15789,Shapewater Changing Butterfly LED Light Desk ...,91748c1bd9e54c79,1624242361
34125,Jam tangan Skmei Kotak keren dan cool 1274,b11c961ecf079c1d,1089805460
24296,Nutrinidrink rasa vanila (READY STOCK),b3f00f0dd60ecc78,899136538
24651,Casing Silikon Matte Frosted Ultra Tipis untuk...,eb61941ed3169ca9,1557471785
31294,Dispenser Sabun Cair 2 in 1 With Key Lock 2 Ta...,f9698c9e86868e39,2641040768
34144,EQUILIBRIO KITTEN 7.5 KG + 750 GR,b939863e96373832,3798354492
27937,Penggaris Besi 20cm (1pc),bfe0c11ff0071cf0,334767061
17225,Cutter Blade Refill Isi Pisau Pemotong Joyko A...,ed07960f960f0d39,1644754067


# save 

In [54]:
pout = f"{HOME}/data/dev0010"
!mkdir -p $pout

In [55]:
np.save(f"{pout}/hdists.npy", hdists)
np.save(f"{pout}/hsims.npy", hsims)

# fin